In [88]:
import pandas as pd
import math

In [89]:
#GET DICTIIONARIES
chords_dict = pd.read_pickle('../Data_prep_for_app/Foreign_Key_Dicts/chord_dict').to_dict()
time_signature_dict = pd.read_pickle('../Data_prep_for_app/Foreign_Key_Dicts/time_signature_dict').to_dict()
composers_dict = pd.read_pickle('../Data_prep_for_app/Foreign_Key_Dicts/composer_dict').to_dict()
lyricists_dict = pd.read_pickle('../Data_prep_for_app/Foreign_Key_Dicts/lyricist_dict').to_dict()
performers_dict = pd.read_pickle('../Data_prep_for_app/Foreign_Key_Dicts/performer_dict').to_dict()
composers_coll_dict = pd.read_pickle('../Data_prep_for_app/Foreign_Key_Dicts/composer_coll_dict').to_dict()
performers_coll_dict = pd.read_pickle('../Data_prep_for_app/Foreign_Key_Dicts/performer_coll_dict').to_dict()
work_dict = pd.read_pickle('../Data_prep_for_app/Foreign_Key_Dicts/work_dict')


altyakj = pd.read_json('../Datasets/jsonFiles/allTheThingsYouAreKeithJarrettSolo.json')
sdmpwckj = pd.read_json('../Datasets/jsonFiles/somedayMyPrinceWillComeKeithJarrettSolo.json')
iiwabkj = pd.read_json('../Datasets/jsonFiles/ifIWereABellKeithJarrettSolo.json')
alkj = pd.read_json('../Datasets/jsonFiles/autumnLeavesKeithJarrettSolo.json')
ghkj = pd.read_json('../Datasets/jsonFiles/groovinHighKeithJarrettSolo.json')
dowarkj = pd.read_json('../Datasets/jsonFiles/daysOfWineAndRosesKeithJarrettSolo.json')
sbskj = pd.read_json('../Datasets/jsonFiles/stellaByStarlightKeithJarrettSolo.json')

notes = pd.concat([altyakj, sdmpwckj, iiwabkj, alkj, ghkj, dowarkj, sbskj]).reset_index(drop=True)
notes["Current chord type"] = notes["Current chord type"].fillna('dom')
notes["Lyricist name"] = notes["Lyricist name"].fillna('No lyricist')

In [90]:
def get_chord_id(data_row):
    c = str(data_row['Current chord root'])  + str(data_row['Current chord type']) + str(data_row['Current chord root as int'])
    return chords_dict[c]
    
def get_ts(data_row):
    x = str(data_row["Time signature numerator"]) + str(data_row["Time signature denominator"])
    return time_signature_dict[x]

def get_composer(data_row):
    x = data_row["Composer name"]
    return composers_dict[x]

def get_lyricist(data_row):
    x = data_row["Lyricist name"]
    return lyricists_dict[x]

def get_performer(data_row):
    x = data_row["Performer name"]
    return performers_dict[x]

def get_performer_collection(data_row):
    x = data_row["Performer collection"]
    return performers_coll_dict[x]

def get_composer_collection(data_row):
    x = data_row["Composer collection"]
    return composers_coll_dict[x]

def get_work(data_row):
    x = data_row["Title"]
    return work_dict[x]

In [91]:
notes['Lyricist name'].value_counts()

Johnny Mercer           2669
Oscar Hammerstein II    2027
Frank Loesser           1983
Larry Morey             1815
No lyricist             1811
Ned Washington          1512
Name: Lyricist name, dtype: int64

In [92]:
notes['chord_symbol'] = notes.apply(get_chord_id, axis=1)
notes['time_signature_id'] = notes.apply(get_ts, axis=1)
notes['composer_id'] = notes.apply(get_composer, axis=1)
notes['lyricist_id'] = notes.apply(get_lyricist, axis=1)
notes['performer_id'] = notes.apply(get_performer, axis=1)
notes['performer_col'] = notes.apply(get_performer_collection, axis=1)
notes['composer_col'] = notes.apply(get_composer_collection, axis=1)
notes['work_id'] = notes.apply(get_work, axis=1)

In [93]:
#notes.head(1).T

In [94]:
#notes.groupby(['work_id', 'Title', 'Lyricist name', 'Composer name', 'Performer collection', 'Composer collection']).count()

In [95]:
del notes['composer_id']
del notes['lyricist_id']
del notes['performer_id']
del notes['performer_col']
del notes['composer_col']

In [96]:
notes = notes[[
 'Midi number',
 'Quarter beats per minute',
 'Duration',
 'Duration due to tied notes',
 'Location',
 'Location in measure',
 'Measure location',
 'Current measure',
 'Harmony note flag',
 'Voice',
 'chord_symbol',
 'time_signature_id',
 'work_id',
 ] 
]

In [97]:
notes['articulation_id'] = 0
notes['dynamic_id'] = 0

In [98]:
notes.columns.tolist()

['Midi number',
 'Quarter beats per minute',
 'Duration',
 'Duration due to tied notes',
 'Location',
 'Location in measure',
 'Measure location',
 'Current measure',
 'Harmony note flag',
 'Voice',
 'chord_symbol',
 'time_signature_id',
 'work_id',
 'articulation_id',
 'dynamic_id']

In [99]:
notes.columns = [
'midi_number',
 'tempo_in_qbm',
 'duration',
 'duration_due_to_tied_notes',
 'location',
 'location_in_measure',
 'starting_location_of_this_measure',
 'current_measure',
 'harmony_note_flag',
 'voice',
 'chord_symbol',
 'time_signature_id',
 'work_id',
 'articulation_id',
 'dynamic_id',]

In [100]:
def convert_midi_number_to_string(data_row):
    midi_number = data_row["midi_number"]
    octave = math.floor(midi_number / 12) - 1
    remainder = midi_number % 12
    note_string = ''
    
    if remainder == 0:
        note_string = 'C'
    elif remainder == 1:
        note_string = 'C#/Db'
    elif remainder == 2:
        note_string = 'D'
    elif remainder == 3:
        note_string = 'D#/Eb'
    elif remainder == 4:
        note_string = 'E'
    elif remainder == 5:
        note_string = 'F'
    elif remainder == 6:
        note_string = 'F#/Gb'
    elif remainder == 7:
        note_string = 'G'
    elif remainder == 8:
        note_string = 'G#/Ab'
    elif remainder == 9:
        note_string = 'A'
    elif remainder == 10:
        note_string = 'A#/Bb'
    elif remainder == 11:
        note_string = 'B'
    
    
    
    if data_row["midi_number"] < 0:
        return 'rest'
    else:
        return note_string + str(octave)
    
    
def convert_midi_number_to_string_without_octave(data_row):
    midi_number = data_row["midi_number"]
    octave = math.floor(midi_number / 12) - 1
    remainder = midi_number % 12
    note_string = ''
    
    if remainder == 0:
        note_string = 'C'
    elif remainder == 1:
        note_string = 'C#/Db'
    elif remainder == 2:
        note_string = 'D'
    elif remainder == 3:
        note_string = 'D#/Eb'
    elif remainder == 4:
        note_string = 'E'
    elif remainder == 5:
        note_string = 'F'
    elif remainder == 6:
        note_string = 'F#/Gb'
    elif remainder == 7:
        note_string = 'G'
    elif remainder == 8:
        note_string = 'G#/Ab'
    elif remainder == 9:
        note_string = 'A'
    elif remainder == 10:
        note_string = 'A#/Bb'
    elif remainder == 11:
        note_string = 'B'
    
    
    
    if data_row["midi_number"] < 0:
        return 'rest'
    else:
        return note_string
    

def convert_float_duration_to_string(data_row):
    if data_row['duration_due_to_tied_notes'] == 240:
        return 'Eighth note'
    elif data_row['duration_due_to_tied_notes'] == 120:
        return "Sixteenth note"
    elif data_row['duration_due_to_tied_notes'] == 160:
        return "Sixteenth note triplet"
    elif data_row['duration_due_to_tied_notes'] == 480:
        return "Half note"
    elif data_row['duration_due_to_tied_notes'] == 960:
        return "Dotted half note"
    elif data_row['duration_due_to_tied_notes'] == 96:
        return "Sixteenth note"
    elif data_row['duration_due_to_tied_notes'] == 80:
        return "Sixteenth note triplet"
    elif data_row['duration_due_to_tied_notes'] == 320:
        return "Quarter note tripet"
    elif data_row['duration_due_to_tied_notes'] == 720:
        return "Half note triplet"
    elif data_row['duration_due_to_tied_notes'] == 1920:
        return "Dotted whole note triplet"
    elif data_row['duration_due_to_tied_notes'] == 137:
        return "Sixteenth note quintuplet"
    elif data_row['duration_due_to_tied_notes'] == 360:
        return "Dotted eigtht note"
    elif data_row['duration_due_to_tied_notes'] == 60:
        return "Thirty second note"
    elif data_row['duration_due_to_tied_notes'] == 1440:
        return "Half note dotted triplet"
    elif data_row['duration_due_to_tied_notes'] == 192:
        return "Sixteenth note dotted triplet"
    elif data_row['duration_due_to_tied_notes'] == 107:
        return "Thirty second note dotted triplet"
    elif data_row['duration_due_to_tied_notes'] == 180:
        return "Dotted half note"
    elif data_row['duration_due_to_tied_notes'] == 40:
        return "Sixteenth note"
    elif data_row['duration_due_to_tied_notes'] == 1680:
        return "Dotted half note"
    else:
        return 'fix'

In [101]:
notes['pitch_name'] = notes.apply(convert_midi_number_to_string, axis=1)
notes['pitch_name_without_octave'] = notes.apply(convert_midi_number_to_string_without_octave, axis=1)
notes['duration_name'] = notes.apply(convert_float_duration_to_string, axis=1)
notes['instrument'] = 0

In [102]:
notes.tail(1).T

,11816
midi_number,-1
tempo_in_qbm,60
duration,960
duration_due_to_tied_notes,960
location,308160
location_in_measure,960
starting_location_of_this_measure,307200
current_measure,161
harmony_note_flag,False
voice,1


In [103]:
notes.to_csv('../Data_prep_for_app/Prepared_tables/note_eventsV2.csv')

In [105]:
notes.tail(1).T

,11816
midi_number,-1
tempo_in_qbm,60
duration,960
duration_due_to_tied_notes,960
location,308160
location_in_measure,960
starting_location_of_this_measure,307200
current_measure,161
harmony_note_flag,False
voice,1
